# Install Dependencies

In [ ]:
%%capture
!pip install monai
!pip install dicom2nifti

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp /content/drive/MyDrive/Machine-Learning-Biomedicine/PankVision-3D/preprocess/final_preprocess.py /content
!cp /content/drive/MyDrive/Machine-Learning-Biomedicine/PankVision-3D/utilities/final_utilities.py /content
!cp /content/drive/MyDrive/Machine-Learning-Biomedicine/PankVision-3D/model/get_model.py /content

In [ ]:
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.losses import DiceLoss, DiceCELoss

import torch
from final_utilities import train
from get_model import get_model
from final_preprocess import preprocess_image, prepare

import numpy as np
from skimage import exposure
from monai.transforms import (
    Compose,
    LoadImaged,
    AddChanneld,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
    Resized,
    ToTensord,
)
from monai.data import CacheDataset, Dataset, DataLoader
from glob import glob
import os
from monai.utils import set_determinism

## Checking for CUDA

In [ ]:
if torch.cuda.is_available():
    print("CUDA is available. You can use the GPU.")
else:
    print("CUDA is not available. You can only use the CPU.")

CUDA is available. You can use the GPU.


## Preprocess Dataset

In [ ]:
data_dir = '/content/drive/MyDrive/Machine-Learning-Biomedicine/PankVision-3D/dataset/dataset-007/Data_Train_Test/'

In [ ]:
data_in = prepare(data_dir, cache=True)

/usr/local/lib/python3.10/dist-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)
/usr/local/lib/python3.10/dist-packages/monai/utils/deprecate_utils.py:111: FutureWarning: <class 'monai.transforms.utility.dictionary.AddChanneld'>: Class `AddChanneld` has been deprecated since version 0.8. It will be removed in version 1.3. please use MetaTensor data type and monai.transforms.EnsureChannelFirstd instead with `channel_dim='no_channel'`.
  warn_deprecated(obj, msg, warning_category)
/usr/local/lib/python3.10/dist-packages/monai/utils/deprecate_utils.py:221: FutureWarning: monai.transforms.utility.dictionary EnsureChannelFirstd.__init__:meta_keys: Argument `meta_keys` has been deprecated since version 0.9. not needed if ima

# Training

In [ ]:
model_dir = '/content/drive/MyDrive/Machine-Learning-Biomedicine/PankVision-3D/results/dataset-007/v6 (dynunet)'

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

args = {
    'model_name': 'DynUNet',
    'pretrained': True,
    'dropout': 0.1
}
model = get_model(args)
model = model.to(device)

# loss_function = nn.CrossEntropyLoss().to(device)
# loss_function = TverskyLoss(alpha=0.7, beta=0.3).to(device)
loss_function = DiceLoss(to_onehot_y=True, softmax=True).to(device)
optimizer = torch.optim.Adam(model.parameters(), 1e-4, weight_decay=1e-5, amsgrad=True)

if __name__ == '__main__':
    train(model, data_in, loss_function, optimizer, 150, model_dir)

Streaming output truncated to the last 5000 lines.
Train_dice: 0.4983
26/222, Train_loss: 0.2566
Train_dice: 0.6569
27/222, Train_loss: 0.2661
Train_dice: 0.6478
28/222, Train_loss: 0.4662
Train_dice: 0.5144
29/222, Train_loss: 0.2728
Train_dice: 0.6419
30/222, Train_loss: 0.3930
Train_dice: 0.5575
31/222, Train_loss: 0.3280
Train_dice: 0.6059
32/222, Train_loss: 0.3454
Train_dice: 0.5852
33/222, Train_loss: 0.3115
Train_dice: 0.6133
34/222, Train_loss: 0.2652
Train_dice: 0.6503
35/222, Train_loss: 0.2947
Train_dice: 0.6232
36/222, Train_loss: 0.2503
Train_dice: 0.6644
37/222, Train_loss: 0.2300
Train_dice: 0.6806
38/222, Train_loss: 0.3326
Train_dice: 0.5998
39/222, Train_loss: 0.2825
Train_dice: 0.6409
40/222, Train_loss: 0.2820
Train_dice: 0.6388
41/222, Train_loss: 0.4255
Train_dice: 0.5350
42/222, Train_loss: 0.2091
Train_dice: 0.7204
43/222, Train_loss: 0.2433
Train_dice: 0.6682
44/222, Train_loss: 0.4804
Train_dice: 0.5053
45/222, Train_loss: 0.3572
Train_dice: 0.5829
46/222, Tr

## Check the Size of Data (Add to main) - For Debugging

In [ ]:
# assuming prepare function has been imported from another file
train_loader, test_loader = prepare(data_dir)

for batch_data in  train_loader:
  volume = batch_data["vol"]
  label = batch_data["seg"]

  # print size of volume and label tensors
  print(f"Volume size: {volume.size()}")
  print(f"Label size: {label.size()}")

  # only process one batch
  break